In [24]:
import tiktoken

def get_num_of_token(string: str, enc: "string"):
    """this returns the number of token in text string"""
    encoding = tiktoken.get_encoding(enc)
    token = len(encoding.encode(string))
    return token


get_num_of_token("Hello world", "cl100k_base")


2

In [25]:
from langchain_ollama import OllamaEmbeddings

questions= "what is javascript"
document = "JavaScript is a high-level, interpreted programming language primarily used for creating interactive web applications. It is dynamically typed, prototype-based, and supports functional, object-oriented, and imperative programming styles."
embeddings = OllamaEmbeddings(model="mxbai-embed-large")



query_result = embeddings.embed_query(questions)
document_result = embeddings.embed_query(document)
print(len(query_result))

1024


In [26]:
import numpy
def cosine_similarity(vec1, vec2):
    by_product = numpy.dot(vec1, vec2)
    norm_vec1 = numpy.linalg.norm(vec1)
    norm_vec2 = numpy.linalg.norm(vec2)

    cosine_sim = by_product / ( norm_vec1 * norm_vec2)
    return cosine_sim

cosine_similarity(query_result, document_result)

np.float64(0.8309821924823665)

In [27]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(web_path="https://www.example.com")
data = loader.load()
print(data)

[Document(metadata={'source': 'https://www.example.com', 'title': 'Example Domain', 'language': 'No language found.'}, page_content='\n\n\nExample Domain\n\n\n\n\n\n\n\nExample Domain\nThis domain is for use in illustrative examples in documents. You may use this\n    domain in literature without prior coordination or asking for permission.\nMore information...\n\n\n\n')]


In [28]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

textSplitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=10, chunk_overlap=2)
splitted_docs = textSplitter.split_documents(data)

In [33]:
from langchain_community.vectorstores import MongoDBAtlasVectorSearch
from pymongo import MongoClient
import certifi
import dotenv
from os import path

env_path = path.expanduser("~"+"/langchain-python/.env")
dotenv.load_dotenv(env_path,encoding="utf-8")
MONGODB_URI = dotenv.get_key(env_path,  "MONGODB_URI")

client = MongoClient(MONGODB_URI, tlsCAFile=certifi.where())
DB_NAME="langchain-test-db"
COLL_NAME="langchain-test-vectorstore"
ATLAS_VECTOR_SEARCH_INDEX_NAME="langchain-test-index"

collection = client[DB_NAME][COLL_NAME]


vector_store = MongoDBAtlasVectorSearch(collection=collection,embedding=embeddings, index_name=ATLAS_VECTOR_SEARCH_INDEX_NAME,relevance_score_fn="cosine")
retriever = vector_store.from_documents(documents=splitted_docs, embedding=embeddings,collection=collection).as_retriever()
print(retriever)

tags=['MongoDBAtlasVectorSearch', 'OllamaEmbeddings'] vectorstore=<langchain_community.vectorstores.mongodb_atlas.MongoDBAtlasVectorSearch object at 0x126613110> search_kwargs={}
